In [ ]:
#Data Prep and Vis
import pandas as pd
import numpy as np

df=pd.read_csv('train.csv')
df.head(5)
y=df['SalePrice']
test=df.drop('SalePrice',axis=1)
df['GarageYrBlt'].fillna('No',inplace=True)
df['LotFrontage'] = df.groupby('Neighborhood').transform(lambda x: x.fillna(x.mean()))
df.info()
df['ExterQual'].value_counts()
#want to write a function to do one way ANOVA and test for normality


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import scipy.stats as st

def R_func(y,X):

    # Prepare the data
    encoder = OneHotEncoder()
    X=X.astype(str)
    X = encoder.fit_transform(X.values.reshape(-1,1))
    #Not use train/test data to match R style calculation

    # Perform regression
    reg = LinearRegression()
    reg.fit(X, y)
    y_pred = reg.predict(X)

    R2 = r2_score(y, y_pred)
    
    return R2

df_r = df.drop([ 'PoolQC','Fence', 'MiscFeature'],axis=1)
cat_attrib= Object(df_r,list(df_r))

for i in range(0,len(cat_attrib)-1):
    r=R_func(df['SalePrice'],df[cat_attrib[i]])
    if r>.2:
        print('var = %s r=%6.3f' % (cat_attrib[i], r))
#or do it r style        
import statsmodels.formula.api as smf
#Find way to do all at once

model = smf.ols(formula='SalePrice ~ C(BsmtQual)+C(KitchenQual)+C(FireplaceQu)', data=df)
results = model.fit()

In [ ]:
def split(arr, size): #splits list into list of smaller lists  - from stack overflow
     arrs = []
     while len(arr) > size:
         piece = arr[:size]
         arrs.append(piece)
         arr   = arr[size:]
     arrs.append(arr)
     return arrs
    
def NonObject(data,arr): #keeps only non-object dtypes in list
    arrs=[]
    for h in arr:
        if data[h].dtype=='object':
            continue
        arrs.append(h)
    return arrs

def Object(data,arr): #keeps only non-object dtypes in list
    arrs=[]
    for h in arr:
        if data[h].dtype=='object':
            arrs.append(h)
        continue
    return arrs


def correlation_matrix(df,titles):
    f, ax = plt.subplots(figsize=(9, 9))
    sns.heatmap(df.corr(), vmax=1, square=True, annot=True, cmap="RdBu_r")

headers=NonObject(test,list(test))
OHeaders=Object(test,list(test))
print(OHeaders)

df_Keeper_Num=['TotalBsmtSF','1stFlrSF', '2ndFlrSF', 'LotFrontage', 'MasVnrArea','GrLivArea','FullBath',
               'TotRmsAbvGrd','GarageCars', 'GarageArea', 'WoodDeckSF']
df_Keeper_Corr=['OverallQual','YearBuilt','YearRemodAdd','MasVnrArea','TotalBsmtSF','1stFlrSF','FullBath','GrLivArea',
            'TotRmsAbvGrd','Fireplaces', 'GarageYrBlt', 'GarageCars','GarageArea']
pair_keepers=split(headers,10) #all numeric variables - 37


for i in range(0,len(pair_keepers)-1):
    plot=pair_keepers[i] 
    plot.extend(['SalePrice'])
    correlation_matrix(df[plot],plot)